## Search data

In [ ]:
import os

# Initialize a set to store file paths
parquet_files = set()

# Define the root directory where the Parquet files are located
root_dir = r'C:\Git\DSI324_wildfire\df_thai'

# Recursively scan the directory and find all Parquet files
print("🔍 Starting directory scan...")
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.parquet'):
            file_path = os.path.join(root, file)
            parquet_files.add(file_path)  # Add the full path of each .parquet file
            print(f"✅ Found Parquet file: {file_path}")

print("🔍 Scan complete!")


## Shape file

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
#adm1 จังหวัด, adm2 อำเภอ
gdf = gpd.read_file(r"C:\Git\DSI324_wildfire\shape_file\tha_admbnda_adm1_rtsd_20220121.shp")

## Plot map

### HeatMap

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from datetime import datetime
from shapely.geometry import Point

# ---------- Step 1: Load Thailand province boundaries ----------
shapefile_path = r"C:\Git\DSI324_wildfire\shape_file\tha_admbnda_adm1_rtsd_20220121.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.drop(columns=gdf.select_dtypes(include=['datetime64']).columns)
gdf = gdf.to_crs(epsg=4326)

# ---------- Step 2: Initialize Folium Map ----------
map_center = [13.7367, 100.5231]  # Bangkok
zoom_level = 6
mymap = folium.Map(location=map_center, zoom_start=zoom_level)

# ---------- Step 3: Prepare Fire Points for HeatMap ----------
heat_data = []

# Filter options
filter_mode = 'range'  # Choose 'exact' or 'range'
filter_date_start = datetime.strptime("2025-04-01", "%Y-%m-%d").date()
filter_date_end = datetime.strptime("2025-04-21", "%Y-%m-%d").date()
filter_date_exact = datetime.strptime("2025-04-02", "%Y-%m-%d").date()

# Assuming parquet_files is already defined somewhere
for file_path in parquet_files:
    try:
        df = pd.read_parquet(file_path)
        df['acq_date'] = pd.to_datetime(df['acq_date']).dt.date

        # Filter by date
        if filter_mode == 'exact':
            df = df[df['acq_date'] == filter_date_exact]
        elif filter_mode == 'range':
            df = df[(df['acq_date'] >= filter_date_start) & (df['acq_date'] <= filter_date_end)]

        # Ensure necessary columns exist
        if 'latitude' in df.columns and 'longitude' in df.columns and not df.empty:
            for _, row in df.iterrows():
                lat = row['latitude']
                lon = row['longitude']
                brightness = row['brightness'] if 'brightness' in row else 1  # Optional weight

                # HeatMap supports optional weight as 4th element
                heat_data.append([lat, lon, brightness])  # You can also use confidence if you want

            print(f"📍 Plotted: {file_path} ({len(df)} points)")
        else:
            pass

    except Exception as e:
        print(f"❌ Failed to process {file_path}: {e}")

# ---------- Step 4: Create Heat Points GeoDataFrame ----------
heat_points = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon, _ in heat_data],  # Using the lat, lon from your heat_data
    crs="EPSG:4326"  # Coordinate system (same as gdf)
)

# ---------- Step 5: Spatial Join - Count Heat Spots per Province ----------
# Perform a spatial join to match heat points to provinces
joined = gpd.sjoin(heat_points, gdf, how="left", predicate="within")

# ---------- Step 6: Count Heat Spots per Province ----------
# Count how many heat points belong to each province
province_counts = joined['ADM1_TH'].value_counts().reset_index()
province_counts.columns = ['ADM1_TH', 'heat_spot_count']

# ---------- Step 7: Merge Heat Spot Count into gdf ----------
gdf = gdf.merge(province_counts, on='ADM1_TH', how='left')

# Fill NaNs with 0 for provinces with no heat spots and convert to integer
gdf['heat_spot_count'] = gdf['heat_spot_count'].fillna(0).astype(int)

# ---------- Step 8: Add Province Boundaries with Heat Spot Count ----------
folium.GeoJson(
    gdf,
    name='Provinces',
    tooltip=folium.GeoJsonTooltip(fields=['ADM1_TH', 'heat_spot_count'], aliases=['จังหวัด', 'จำนวนจุดความร้อน']),
    style_function=lambda x: {
        'fillColor': '#e6bead',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.1
    }
).add_to(mymap)

# ---------- Step 9: Add HeatMap ----------
min_b, max_b = 250, 400
normalized_heat_data = []

for lat, lon, brightness in heat_data:
    b = max(min_b, min(brightness, max_b))
    weight = (b - min_b) / (max_b - min_b)
    normalized_heat_data.append([lat, lon, weight])

gradient = {
    "0.2": "#FFA500",  
    "0.5": "#FF4500",  
    "0.8": "#FF0000",  
    "1.0": "#8B0000"   
}

if normalized_heat_data:
    HeatMap(
        normalized_heat_data,
        radius=10,
        blur=15,
        max_zoom=7,
        gradient=gradient
    ).add_to(mymap)

# ---------- Step 10: Create the Table as HTML String ----------
table_html = '<table border="1" style="width:100%; font-family: Arial, sans-serif; font-size: 14px; text-align: left;">'
table_html += '<tr><th>Province</th><th>Heat Spot Count</th></tr>'

for idx, row in province_counts.iterrows():
    table_html += f'<tr><td>{row["ADM1_TH"]}</td><td>{row["heat_spot_count"]}</td></tr>'

table_html += '</table>'

# ---------- Step 11: Embed Table in HTML with Map ----------
html_content = mymap._repr_html_()  # Get the HTML representation of the map

# Add the table and fixed styling for the table's position
html_content = f'''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Thailand Heat Spot Map</title>
    <style>
        /* Map styling */
        #map {{
            width: 100%;
            height: 100vh;
        }}

        /* Table styling for fixed position */
        #heat_table {{
            position: fixed;
            top: 10px;
            right: 10px;
            width: 250px;
            height: 400px;
            padding: 10px;
            background-color: rgba(255, 255, 255, 0.8);
            border: 1px solid #ccc;
            overflow-y: auto;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            z-index: 1000;
        }}
    </style>
</head>
<body>
    <div id="map">{html_content}</div>

    <!-- Heat Spot Table -->
    <div id="heat_table">
        {table_html}
    </div>
</body>
</html>
'''

# ---------- Step 12: Save the Final HTML File with Map and Table ----------
with open("map_with_heatmap.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("✅ Saved as 'map_with_heatmap.html'")
